<a href="https://colab.research.google.com/github/bilal-rachik/detection-in-contract-management/blob/master/object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip install --upgrade --force-reinstall tensorflow-gpu==1.14

In [0]:
!pip install numpy==1.17

**Clonez le référentiel de modèles TensorFlow:**

In [0]:
!git clone --quiet https://github.com/tensorflow/models.git

**Compilation de Protobuf**
L'API de détection d'objets Tensorflow utilise Protobufs pour configurer le modèle et les paramètres de formation. Avant que le framework puisse être utilisé, les bibliothèques Protobuf doivent être compilées. Pour ce faire, exécutez la commande suivante à partir du répertoire / models / research /:


In [0]:
!pip install pycocotools

In [0]:
%cd /content/models/research

In [0]:
!protoc object_detection/protos/*.proto --python_out=.

Ajouter des bibliothèques à PYTHONPATH Lors de l'exécution locale, les répertoires tensorflow / models / research / et slim doivent être ajoutés à PYTHONPATH. Cela peut être fait en exécutant ce qui suit à partir de tensorflow / models / research /:

In [0]:
import os
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim'

In [0]:
%%bash 
pip install .

# Testez votre installation


In [0]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [0]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile



Par défaut, nous utilisons ici un modèle "SSD avec Mobilenet". Consultez le zoo des modèles de détection pour obtenir une liste des autres modèles pouvant être exécutés immédiatement avec des vitesses et des précisions variables.
# Loader

In [0]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))
  model = model.signatures['serving_default']

  return model



# Chargement de la carte d'étiquettes

In [0]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

nous allons tester sur 2 images:

In [0]:
import pathlib
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

# Detection

Charger un modèle de détection d'objet:

In [0]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)


Vérifiez la signature d'entrée du modèle, il attend un lot d'images en 3 couleurs de type uint8:

In [0]:
print(detection_model.inputs)

Ajoutez une fonction wrapper pour appeler le modèle et nettoyez les sorties:

In [0]:

def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  output_dict = model(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict


Exécutez-le sur chaque image de test et affichez les résultats:

In [0]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [0]:
for image_path in TEST_IMAGE_PATHS:
  show_inference(detection_model, image_path)

# Génération de TFRecords pour la formation

Charger le dossier de workspace dans mon dépôt git sur votre drive


In [0]:
workspace
  | training_Faster_RCNN
  │ ├─ annotations
  │ ├─ images
  │ │     ├─ test
  │ │     └─ train
  │ ├─ pre-trained-model
  │ ├─ training
  │ └─ README.md
  ├─ xml_to_csv.py
  ├─ partition_dataset.py
  ├─ make_official_dataset.py
  │─generate_tfrecord.py


L'API de détection d'objets TensorFlow ne prend pas les fichiers json ou xml en entrée, mais elle a besoin des fichiers TFRecord format pour former le modèle.

Pour ce faire, cela depend de format que vous avez exportés vos actifs dans VOTT.

* 1 VoTT (generic JSON schema) : il faut utiliser le script make_official_dataset.py Le make_official_dataset.py importe ..training_Faster_RCNN/images/detection-export.json où toutes les annotations de bbox sont stockées, et crée un ensemble de train et de test TFRrecords.

Pour utiliser le script, en supposant que vous avez toutes vos images et detection-export.json fichier à l'intérieur training_Faster_RCNN\images, exécutez simplement la commande suivante:





In [0]:
! cd /content/drive/My\ Drive/my-project/workspace

In [0]:
python3 make_official_dataset.py --labels "signature" "Handwrit" "intial" --img_dir [PATH_TO_IMAGES_FOLDER]/images 

**format Pascal VOC**: il y a deux étapes:

    *Conversion des *.xml fichiers individuels en un *.csv fichier unifié pour chaque ensemble de données.
    
    *Conversion des *.csv fichiers de chaque jeu de données en *.record fichiers (format TFRecord).

Copiez maintenant 80% de votre fichier image dans le dossier images / train et  20% dans votre dossier de test

Pour le  faire , en supposant que vous avez toutes vos images et *.xml fichiers à l'intérieur training_Faster_RCNN\images, exécutez simplement la commande suivante:


In [0]:
! python partition_dataser.py -x -i training_Faster_RCNN\images -r 0.2

## Maintenant, dans ce dossier, nous pouvons transformer vos fichiers XML en train_label.csv et test_label.csv en ouvrant la ligne de commande et en tapant:

In [0]:
##Create train data**:
!python xml_to_csv.py -i /content/drive/My\ Drive/my-project/workspace/training_faster_rcnn/images/train -o /content/drive/My\ Drive/my-project/workspace/training_faster_rcnn/annotations/train_labels.csv

##Create test data:
!python xml_to_csv.py -i /content/drive/My\ Drive/my-project/workspace/training_faster_rcnn/images/test -o /content/drive/My\ Drive/my-project/workspace/training_faster_rcnn/annotations/test_labels.csv

**générer les TFRecords en tapant**

In [0]:
!python generate_tfrecord.py  --csv_input=/content/drive/My\ Drive/my-project/workspace/training_faster_rcnn/annotations/train_labels.csv --img_path=/content/drive/My\ Drive/my-project/workspace/training_faster_rcnn/images/train  --output_path=/content/drive/My\ Drive/my-project/workspace/training_faster_rcnn/annotations/train.record

# Create test data:
!python generate_tfrecord.py --csv_input=/content/drive/My\ Drive/my-project/workspace/training_faster_rcnn/annotations/test_labels.csv --img_path=/content/drive/My\ Drive/my-project/workspace/training_faster_rcnn/images/test --output_path=/content/drive/My\ Drive/my-project/workspace/training_faster_rcnn/annotations/test.record

# Téléchargez le modèle Faster-RCNN-Inception-V2-COCO depuis le zoo modèle de TensorFlow¶

In [0]:
#From training_Faster_faster_rcnn\pre-trained-model
%%bash 
cd /content/drive/My\ Drive/my-project/workspace/training_faster_rcnn/pre-trained-model
wget http://download.tensorflow.org/models/object_detection/faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
tar -zxvf faster_rcnn_inception_v2_coco_2018_01_28.tar.gz
rm -r faster_rcnn_inception_v2_coco_2018_01_28.tar.gz

## Former le modèle 

In [0]:
%%bash 
cd /content/drive/My\ Drive/my-project/workspace/training_faster_rcnn
python model_main.py --alsologtostderr --model_dir=training/ --pipeline_config_path=/content/drive/My\ Drive/my-project/workspace/training_faster_rcnn/training/faster_rcnn_inception_v2_pets.config


# Surveiller la progression de formation à l'aide de TensorBoard

In [0]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [0]:
%cd /content/drive/My\ Drive/my-project/workspace/training_faster_rcnn

In [0]:
%tensorboard --logdir=training/

In [0]:
import os
import subprocess
import json
import time
import requests
def _get_ngrok_tunnel():
    while True:
        try:
            tunnels_json = requests.get("http://localhost:4040/api/tunnels").content
            public_url = json.loads(tunnels_json)['tunnels'][0]['public_url']
            return public_url
        except Exception:
            print("Can't get public url, retrying...")
            time.sleep(2)


def _warmup_ngrok_tunnel(public_url):
    while requests.get(public_url).status_code >= 500:
        print("Tunnel is not ready, retrying...")
        time.sleep(2)


def expose_port_on_colab(port):
    os.system("apt-get install net-tools")
    # check that port is open
    while not (":{} ".format(port) in str(subprocess.check_output("netstat -vatn", shell=True))):
        print("Port {} is closed, retrying...".format(port))
        time.sleep(2)

    # run ngrok
    os.system("wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip")
    os.system("unzip ngrok-stable-linux-amd64.zip")
    os.system("./ngrok http {0} &".format(port))
    public_url = _get_ngrok_tunnel()
    _warmup_ngrok_tunnel(public_url)

    print("Open {0} to access your {1} port".format(public_url, port))

In [0]:
%cd /content/drive/My\ Drive/my-project/workspace/training_faster_rcnn

In [0]:
import os
os.system("tensorboard --logdir=training/ --host 0.0.0.0 --port 5007 &")
expose_port_on_colab(5007)

# Exportation d'un graphique d'inférence formé  Exporter le graphique d'inférence

In [0]:
%cd /content/drive/My\ Drive/my-project/workspace/training_faster_rcnn

In [0]:
!python export_inference_graph.py --input_type image_tensor --pipeline_config_path training/faster_rcnn_inception_v2_pets.config --trained_checkpoint_prefix training/model.ckpt-200000 --output_directory trained-inference-graphs/output_inference_graph_v1.pb

In [0]:
# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys

In [0]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [0]:
# Path to frozen detection graph .pb file, which contains the model that is used
PATH_TO_CKPT = '/content/drive/My Drive/my-project/workspace/training_faster_rcnn/trained-inference-graphs/output_inference_graph_v1.pb/frozen_inference_graph.pb'

# Path to label map file
PATH_TO_LABELS = '/content/drive/My Drive/my-project/workspace/training_faster_rcnn/training/label_map.pbtxt'

# Path to image
PATH_TO_IMAGE ='/content/drive/My Drive/my-project/workspace/training_faster_rcnn/images/LMD09443-SLA-5-_Z-1.jpg'


In [0]:
# Load the label map.
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [0]:
# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)


In [0]:
# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

In [0]:
# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value
image = cv2.imread(PATH_TO_IMAGE)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
image_expanded = np.expand_dims(image_rgb, axis=0)

# Perform the actual detection by running the model with the image as input
(boxes, scores, classes, num) = sess.run(
    [detection_boxes, detection_scores, detection_classes, num_detections],
    feed_dict={image_tensor: image_expanded})

# Draw the results of the detection (aka 'visulaize the results')

vis_util.visualize_boxes_and_labels_on_image_array(
    image,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=8,
    min_score_thresh=0.60)


In [0]:
display(Image.fromarray(image))

In [0]:
cv2.imwrite('/content/drive/My Drive/my-project/workspace/training_faster_rcnn/images/out-put.jpg', image)